# Cleaning the Spray Dataset

## Importing modules and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline
sns.set_style('darkgrid')
sns.set_palette('viridis')

spray = pd.read_csv('../data/spray.csv')

In [2]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


## Checking for null values

In [3]:
spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

Since there are so many missing values for time and we have no way to determine the true values, we will drop the time column.

## Checking for other errors

### Duplicated locations

We noticed that one latitude/longitude pair was duplicated over 500 times and one other was duplicated once. We address these issues below:

In [4]:
order_groups = spray.groupby(['Latitude', 'Longitude'], as_index=False).count().sort_values('Date', ascending=False)

In [5]:
order_groups.head()

,Latitude,Longitude,Date,Time
11853,41.986460,-87.794225,541,541
11499,41.983917,-87.793088,2,2
0,41.713925,-87.615892,1,1
9533,41.959113,-87.719752,1,1
9522,41.959028,-87.728890,1,1


In [6]:
order_groups.loc[11499, :]['Latitude']

41.9839166666667

In [7]:
lat_long_count = spray.drop('Time', axis=1).groupby(['Latitude', 'Longitude']).agg(['count'])

lat_long_count.columns

lat_long_count = pd.Series(lat_long_count.values.reshape(1,-1)[0], lat_long_count.index)

lat_long_count.sort_values(ascending=False).head()

Latitude   Longitude 
41.986460  -87.794225    541
41.983917  -87.793088      2
41.894413  -87.710262      1
41.894380  -87.772148      1
41.894343  -87.760688      1
dtype: int64

Through manual inspection, we have determined a spray entry which has been duplicated 541 times.  We are going to remove the duplicates from this list.

In [8]:
mask = (spray['Latitude'] == 41.986460) & (spray['Longitude'] == -87.794225)
spray[mask].shape

(541, 4)

In [9]:
spray[mask].index[1:]

Int64Index([ 490,  491,  492,  493,  494,  495,  496,  497,  498,  499,
            ...
            1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029],
           dtype='int64', length=540)

In [10]:
spray.drop(spray[mask].index[1:], inplace=True)

In [11]:
lat_long_count = spray.drop('Time', axis=1).groupby(['Latitude', 'Longitude']).agg(['count'])

lat_long_count = pd.Series(lat_long_count.values.reshape(1,-1)[0], lat_long_count.index)

lat_long_count.sort_values(ascending=False).head()

Latitude   Longitude 
41.983917  -87.793088    2
42.395983  -88.095757    1
41.894157  -87.754473    1
41.894380  -87.772148    1
41.894343  -87.760688    1
dtype: int64

In [12]:
mask = (spray['Longitude'] == -87.7930883333333) & (spray['Latitude'] ==  41.9839166666667)
sum(mask)

2

In [13]:
spray[mask].index[1:]

Int64Index([485], dtype='int64')

In [14]:
spray.drop(spray[mask].index[1:], inplace=True)

In [15]:
lat_long_count = spray.drop('Time', axis=1).groupby(['Latitude', 'Longitude']).agg(['count'])

lat_long_count = pd.Series(lat_long_count.values.reshape(1,-1)[0], lat_long_count.index)

lat_long_count.sort_values(ascending=False).head()

Latitude   Longitude 
42.395983  -88.095757    1
41.894160  -87.767937    1
41.894402  -87.704128    1
41.894380  -87.772148    1
41.894343  -87.760688    1
dtype: int64

In [16]:
spray.drop('Time', axis=1, inplace=True)

**Now there are no duplicated latitude/longitude pairs and we have dropped the time column.**

## Adding an indicator if a trap is nearby spray zone

In [17]:
train = pd.read_csv('../data/train_cleaned.csv')

In [18]:
from math import sin, cos, radians, asin, sqrt
def global_distance(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956
    
    return c * r

In [19]:
train[['longitude','latitude']].head()

,longitude,latitude
0,-87.800991,41.954690
1,-87.800991,41.954690
2,-87.769279,41.994991
3,-87.824812,41.974089
4,-87.824812,41.974089


In [20]:
spray[['Longitude', 'Latitude']].head()

,Longitude,Latitude
0,-88.089163,42.391623
1,-88.089163,42.391348
2,-88.089157,42.391022
3,-88.089158,42.390637
4,-88.088858,42.390410


In [21]:
stations = {}
for index, row in train.iterrows():
    stations[row['trap']] = (row['longitude'], row['latitude'])

In [22]:
stations

{'T002': (-87.800991, 41.95469),
 'T007': (-87.769279, 41.994991),
 'T015': (-87.824812, 41.974089),
 'T045': (-87.666455, 41.9216),
 'T046': (-87.65449100000002, 41.891118),
 'T048': (-87.654224, 41.867108),
 'T049': (-87.655232, 41.896282),
 'T050': (-87.694259, 41.919343),
 'T054': (-87.632085, 41.921965),
 'T086': (-87.676709, 41.688324),
 'T091': (-87.64886, 41.862292),
 'T094': (-87.666014, 41.720848),
 'T096': (-87.677512, 41.731922),
 'T129': (-87.61156, 41.891126),
 'T143': (-87.795585, 41.999129),
 'T148': (-87.68776899999997, 42.01743),
 'T153': (-87.760886, 41.907645),
 'T159': (-87.649642, 41.732984),
 'T009': (-87.862995, 41.99247800000001),
 'T011': (-87.832763, 41.944869),
 'T016': (-87.81150600000002, 42.011601),
 'T019': (-87.69845699999998, 41.94983),
 'T025': (-87.750938, 41.979243),
 'T028': (-87.689778, 41.986921),
 'T031': (-87.763416, 41.801498),
 'T033': (-87.701431, 41.906638),
 'T089': (-87.64997, 41.723195),
 'T090': (-87.666901, 41.868077),
 'T092': (-87.62

In [24]:
station_distances = {}
for station in stations:
    lon = stations[station][0]
    lat = stations[station][1]
    
    for index, spray_row in spray.iterrows():
        tmp_dist = global_distance(lon, lat, spray_row['Longitude'], spray_row['Latitude'])
        if station in station_distances:
            station_distances[station] = min(tmp_dist, station_distances[station])
        else:
            station_distances[station] = tmp_dist


In [31]:
train.to_csv('../data/train_cleaned_spray.csv', index=False)

## Exporting file

In [32]:
spray.to_csv('../data/spray_cleaned.csv', index=False)